## Exercise 4: Data Warehouse Querying and Basic Geospatial Operations

Skills: 
* Query data warehouse table
* Use dictionary to map values

References: 
* https://docs.calitp.org/data-infra/analytics_new_analysts/05-spatial-analysis-basics.html
* https://docs.calitp.org/data-infra/analytics_new_analysts/06-spatial-analysis-intro.html
* https://docs.calitp.org/data-infra/analytics_new_analysts/07-spatial-analysis-intermediate.html
* https://github.com/jorisvandenbossche/geopandas-tutorial

In [32]:
#gpd -- gdf, pd -- df(can change the name, like stops, or counties)
import geopandas as gpd
import pandas as pd
import os

#os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)
pd.set_option("display.max_rows", 20)

#from calitp_data_analysis.tables import tbls
#from calitp_data_analysis.sql import query_sql
from siuba import *
FOLDER = "./data/"
FILE_NAME = "exercise_4_stops_sample.parquet"
stops=pd.read_parquet(f"{FOLDER}{FILE_NAME}")

## Query a table, turn it into a gdf

You will query the warehouse table for 2 operators, Caltrain and Merced. A `feed_key` is a hash identifier, there's no real meaning to it, but it uniquely identifies a feed for that day.

The `feed_key` values for those 2 operators for 6/1/2022 are provided. 

* Query `mart_gtfs.dim_stops`
* Filter to the feed keys of interest
* Select these columns: `feed_key`, `stop_id`, `stop_lat`, `stop_lon`, `stop_name`
* Return as a dataframe using `collect()`
* Turn the point data into geometry with `geopandas`: [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html)

In [2]:
FEEDS = [
    "25c6505166c01099b2f6f2de173e20b9", # Caltrain
    "52639f09eb535f75b33d2c6a654cb89e", # Merced
]

stops = pd.read_parquet('./data/exercise_4_stops_sample.parquet')
stops2 = (stops
    #tbls.mart_gtfs.dim_stops()
    >> filter(_.feed_key.isin(FEEDS))
    >> select(_.feed_key, _.stop_id, 
             _.stop_lat, _.stop_lon, _.stop_name)
    >> arrange(_.feed_key, _.stop_id, 
               _.stop_lat, _.stop_lon)
    #>> collect() 
)

In [3]:
stops.head()

,feed_key,stop_id,stop_lat,stop_lon,stop_name
0,52639f09eb535f75b33d2c6a654cb89e,768319,37.067249,-120.855585,G St @ Davita Dialysis (southbound)
1,52639f09eb535f75b33d2c6a654cb89e,768398,37.345519,-120.608472,Atwater Transpo
2,25c6505166c01099b2f6f2de173e20b9,san_carlos,37.508033,-122.260200,San Carlos
3,52639f09eb535f75b33d2c6a654cb89e,768199,37.382422,-120.723304,Main St @ F St (northbound)
4,52639f09eb535f75b33d2c6a654cb89e,768413,37.353220,-120.613861,Winton @ Grove (southbound)


In [4]:
#Turn the point data into geometry with geopandas???????

stops_gdf = gpd.GeoDataFrame(
    stops, 
    geometry=gpd.points_from_xy(stops['stop_lon'], stops['stop_lat']),
    crs='EPSG:4326'
)

In [5]:
stops_gdf.head(2)

,feed_key,stop_id,stop_lat,stop_lon,stop_name,geometry
0,52639f09eb535f75b33d2c6a654cb89e,768319,37.067249,-120.855585,G St @ Davita Dialysis (southbound),POINT (-120.85559 37.06725)
1,52639f09eb535f75b33d2c6a654cb89e,768398,37.345519,-120.608472,Atwater Transpo,POINT (-120.60847 37.34552)


In [6]:
stops_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Use a dictionary to map values

* Create a new column called `operator` where `feed_key` is associated with its operator name.
* First, write a function to do it.
* Then, use a dictionary to do it (create new column called `agency`).
* Double check that `operator` and `agency` show the same values. Use `assert` to check.
    * `assert df.operator == df.agency`
* Hint: https://docs.calitp.org/data-infra/analytics_new_analysts/02-data-analysis-intermediate.html

In [7]:
stops.feed_key.value_counts()

52639f09eb535f75b33d2c6a654cb89e    197
25c6505166c01099b2f6f2de173e20b9     35
Name: feed_key, dtype: int64

In [8]:
def name1(row):
    if row.feed_key == '52639f09eb535f75b33d2c6a654cb89e':
        return 'a'
    elif row.feed_key == '25c6505166c01099b2f6f2de173e20b9':
        return 'b'
    else:
        return 'other'
stops['operator'] = stops.apply(name1, axis = 1)

In [9]:
#using a dictionary
name2={'52639f09eb535f75b33d2c6a654cb89e': 'a', '25c6505166c01099b2f6f2de173e20b9': 'b'}

stops['agency'] = stops.feed_key.map(name2)

In [10]:
#two methods got the same answer. do i have to type 'assert' in the front???
stops.operator == stops.agency

0      True
1      True
2      True
3      True
4      True
       ... 
227    True
228    True
229    True
230    True
231    True
Length: 232, dtype: bool

In [11]:
#another way to do this
assert(stops.operator == stops.agency).all()

## Turn lat/lon into point geometry

* There is a [function in shared_utils](https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/geography_utils.py#L170-L192) that does it. Show the steps within the function (the long way), and also create the `geometry` column using `shared_utils`.
* Use `geography_utils.create_point_geometry??` to see what goes into that function, and what that function looks like under the hood.

# Rename columns
stops.rename(columns = {'stop_lon': 'longitude', 'stop_lat':'latitude'}, inplace=True)

# Create geometry column
stops_geo = gpd.points_from_xy(stops.longitude, stops.latitude, crs="EPSG:4326")
stops['geometry2']=stops_geo


#another way to do this
geometry = gpd.points_from_xy(stops.stop_lon, stops.stop_lat, crs="EPSG:4326")

#Another way
# Default CRS for stop_lon, stop_lat is WGS84
stops = stops.assign(geometry=gpd.points_from_xy(df[stop_lon], df[stop_lat], crs="EPSG:4326")

# Another way: Use geography_utils.create_point_geometry?? to see what goes into that function,
#and what that function looks like under the hood.
def create_point_geometry(
    stops: pd.DataFrame,
    longitude_col: str = "stop_lon",
    latitude_col: str = "stop_lat",
    crs: str = crs="EPSG:4326",
) -> gpd.GeoDataFrame

Basic stuff about a geodataframe.

A gdf would have a coordinate reference system that converts the points or lines into a place on the spherical Earth. The most common CRS is called `WGS 84`, and its code is `EPSG:4326`. This is what you'd see when you use Google Maps to find lat/lon of a place.

[Read](https://desktop.arcgis.com/en/arcmap/latest/map/projections/about-geographic-coordinate-systems.htm) about the `WGS 84` geographic coordinate system.

[Read](https://desktop.arcgis.com/en/arcmap/latest/map/projections/about-projected-coordinate-systems.htm) about projected coordinate reference systems, which is essentially about flattening our spherical Earth into a 2D plane so we can measure distances and whatnot.

* Is it a pandas dataframe or a geopandas geodataframe?: `type(gdf)`
* Coordinate reference system: `gdf.crs`
* gdfs must have a geometry column. Find the name of the column that is geometry: `gdf.geometry.name`
* Project the coordinate reference system to something else: `gdf = gdf.to_crs("EPSG:2229")` and check.

In [12]:
stops_gdf.geometry.name

'geometry'

In [ ]:
#gdf = gdf.to_crs("EPSG:2229")

In [ ]:
#gdf.crs

* This GitHub repo has several `geopandas` tutorials that covers basic spatial concepts: https://github.com/jorisvandenbossche/geopandas-tutorial. 
* Skim through the notebooks to see some of the concepts demonstrated, although to actually run the notebooks, you can click on `launch binder` in the repo's README to do so.

## Spatial Join (which points fall into which polygon)

This URL gives you CA county boundaries: https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-county-boundaries/explore?location=37.246136%2C-119.002032%2C6.12

* Go to "I want to use this" > View API Resources > copy link for geojson
* Read in the geojson with `geopandas` and make it a geodataframe: `gpd.read_file(LONG_URL_PATH)`
* Double check that the coordinate reference system is the same for both gdfs using `gdf.crs`. If not, change it so they are the same.
* Spatial join stops to counties: [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html)
    * Play with inner join or left join, what's the difference? Which one do you want?
    * Play with switching around the left_df and right_df, what's the right order?
* By county: count number of stops and stops per sq_mi.
    * Hint 1: Start with a CRS with units in feet or meters, then do a conversion to sq mi. [CRS in shared_utils](https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/geography_utils.py)
    * Hint 2: to find area, you can create a new column and calculate `gdf.geometry.area`. [geometry manipulations docs](https://geopandas.org/en/stable/docs/user_guide/geometric_manipulations.html)

In [13]:
counties = gpd.read_file('https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_County_Boundaries/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')

In [14]:
counties.head(2)

,OBJECTID,COUNTY_NAME,COUNTY_ABBREV,COUNTY_NUM,COUNTY_CODE,COUNTY_FIPS,ISLAND,Shape__Area,Shape__Length,GlobalID,geometry
0,1,Alameda,ALA,1,01,001,NaN,3.402787e+09,308998.650766,e6f92268-d2dd-4cfb-8b79-5b4b2f07c559,"POLYGON ((-122.27125 37.90503, -122.27024 37.9..."
1,2,Alpine,ALP,2,02,003,NaN,3.146939e+09,274888.492411,870479b2-480a-494b-8352-ad60578839c1,"POLYGON ((-119.58667 38.71420, -119.58653 38.7..."


In [ ]:
# If the CRS is not set after checking it with gdf.crs

#gdf = gdf.set_crs('EPSG:4326')


In [15]:
counties.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
#counties = counties.set_crs('EPSG:4326')

In [16]:
counties = counties.to_crs('EPSG:4326')

In [17]:
counties.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [18]:
merge = gpd.sjoin(counties, stops_gdf, how = 'inner', predicate = 'intersects')

In [ ]:
#merge1 = pd.merge(counties, stops, on = 'geometry',
    #how = 'inner', validate = 'm:1')

In [ ]:
#merge2 = pd.merge(counties, stops, left_on = 'COUNTY_NAME',
    #right_on = 'geometry', how = 'left', validate = 'm:1')

In [ ]:
#merge3 = gpd.sjoin(counties, stops, how='left', predicate='intersects', lsuffix='left', rsuffix='right')

In [ ]:
#merge4 = pd.merge(gdf, df, left_on = 'District', right_on = 'CD')

In [19]:
merge.head(2)

,OBJECTID,COUNTY_NAME,COUNTY_ABBREV,COUNTY_NUM,COUNTY_CODE,COUNTY_FIPS,ISLAND,Shape__Area,Shape__Length,GlobalID,geometry,index_right,feed_key,stop_id,stop_lat,stop_lon,stop_name
23,24,Merced,MER,24,24,047,NaN,8.085829e+09,432954.410428,f3d6231c-c7fa-4340-b03f-23c1d0572979,"POLYGON ((-120.37071 37.60543, -120.35414 37.5...",112,52639f09eb535f75b33d2c6a654cb89e,782479,36.964503,-120.653529,Kwik Serv (South Dos Palos)
23,24,Merced,MER,24,24,047,NaN,8.085829e+09,432954.410428,f3d6231c-c7fa-4340-b03f-23c1d0572979,"POLYGON ((-120.37071 37.60543, -120.35414 37.5...",87,52639f09eb535f75b33d2c6a654cb89e,782486,36.981746,-120.627963,Golden Gate Ave & Valeria St Stop


#By county: count number of stops and stops per sq_mi.
#Start with a CRS with units in feet or meters, then do a conversion to sq mi.
WGS84 = "EPSG:4326"
CA_StatePlane = "EPSG:2229"  # units are in feet
CA_NAD83Albers = "EPSG:3310"  # units are in meters

SQ_MI_PER_SQ_M = 3.86 * 10**-7
FEET_PER_MI = 5_280
SQ_FT_PER_SQ_MI = 2.788 * 10**7


In [20]:
counties1 = counties.assign(geometry=counties.geometry.to_crs('EPSG:2229'))

In [21]:
counties1['county_sq_ft'] = counties1.geometry.area

In [22]:
counties1.head(2)

,OBJECTID,COUNTY_NAME,COUNTY_ABBREV,COUNTY_NUM,COUNTY_CODE,COUNTY_FIPS,ISLAND,Shape__Area,Shape__Length,GlobalID,geometry,county_sq_ft
0,1,Alameda,ALA,1,01,001,NaN,3.402787e+09,308998.650766,e6f92268-d2dd-4cfb-8b79-5b4b2f07c559,"POLYGON ((5327843.636 3270649.517, 5328125.534...",2.297674e+10
1,2,Alpine,ALP,2,02,003,NaN,3.146939e+09,274888.492411,870479b2-480a-494b-8352-ad60578839c1,"POLYGON ((6107872.113 3543254.346, 6107910.593...",2.075032e+10


In [23]:
SQ_FT_PER_SQ_MI = 2.788 * 10**7

In [24]:
counties1['county_sq_mi'] = counties1['county_sq_ft']/SQ_FT_PER_SQ_MI

In [25]:
counties1.head(2)

,OBJECTID,COUNTY_NAME,COUNTY_ABBREV,COUNTY_NUM,COUNTY_CODE,COUNTY_FIPS,ISLAND,Shape__Area,Shape__Length,GlobalID,geometry,county_sq_ft,county_sq_mi
0,1,Alameda,ALA,1,01,001,NaN,3.402787e+09,308998.650766,e6f92268-d2dd-4cfb-8b79-5b4b2f07c559,"POLYGON ((5327843.636 3270649.517, 5328125.534...",2.297674e+10,824.129820
1,2,Alpine,ALP,2,02,003,NaN,3.146939e+09,274888.492411,870479b2-480a-494b-8352-ad60578839c1,"POLYGON ((6107872.113 3543254.346, 6107910.593...",2.075032e+10,744.272718


In [26]:
stops_per_county=merge.groupby(['COUNTY_NAME']).agg({'stop_id':'count'}).reset_index()

In [27]:
final_stops = pd.merge(counties1, stops_per_county, on = 'COUNTY_NAME',
    how = 'inner', validate = 'm:1')

In [28]:
final_stops.head(2)

,OBJECTID,COUNTY_NAME,COUNTY_ABBREV,COUNTY_NUM,COUNTY_CODE,COUNTY_FIPS,ISLAND,Shape__Area,Shape__Length,GlobalID,geometry,county_sq_ft,county_sq_mi,stop_id
0,24,Merced,MER,24,24,047,NaN,8.085829e+09,432954.410428,f3d6231c-c7fa-4340-b03f-23c1d0572979,"POLYGON ((5874128.470 3143301.015, 5878700.520...",5.522343e+10,1980.754371,195
1,41,San Mateo,SMT,41,41,081,NaN,2.272024e+09,296746.203111,b4f387b3-07f1-43a1-b18a-a6d81c17e28b,"POLYGON ((5289433.313 3200486.689, 5294453.059...",1.541602e+10,552.941880,17


In [29]:
final_stops['stops_per_sqm'] = final_stops.stop_id/final_stops.county_sq_mi

In [30]:
final_stops.head(2)

,OBJECTID,COUNTY_NAME,COUNTY_ABBREV,COUNTY_NUM,COUNTY_CODE,COUNTY_FIPS,ISLAND,Shape__Area,Shape__Length,GlobalID,geometry,county_sq_ft,county_sq_mi,stop_id,stops_per_sqm
0,24,Merced,MER,24,24,047,NaN,8.085829e+09,432954.410428,f3d6231c-c7fa-4340-b03f-23c1d0572979,"POLYGON ((5874128.470 3143301.015, 5878700.520...",5.522343e+10,1980.754371,195,0.098447
1,41,San Mateo,SMT,41,41,081,NaN,2.272024e+09,296746.203111,b4f387b3-07f1-43a1-b18a-a6d81c17e28b,"POLYGON ((5289433.313 3200486.689, 5294453.059...",1.541602e+10,552.941880,17,0.030745


#to find area, you can create a new column and calculate gdf.geometry.area
gdf.geometry.area

In [ ]:
#counties.geometry.area

In [31]:
counties.geometry.area

/tmp/ipykernel_177/1157395454.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties.geometry.area


0     0.217411
1     0.198471
2     0.161961
3     0.455899
4     0.275908
        ...   
64    0.000059
65    0.000174
66    0.000041
67    0.014070
68    0.018707
Length: 69, dtype: float64